# Mini Project 3 Bayesian

By Lars Magne Stangeland (Student ID: ) and Haavard Fossdal (Student ID: 3040928803)

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim.downloader as api
import math as Math
import os


## Twenty-Questions: Unconstrained Questions

In [ ]:
# Load the CSV file into a DataFrame
file_Path = '../data/unigram_freq.csv'
df = pd.read_csv(file_Path, keep_default_na=False)
count = 0


# trim to first 3000 rows
df = df.sort_values(by='count', ascending=False)
df = df.head(3000)

#Build the Huffman tree
def get_huffman_code(df):

    # Dictionary to store the Huffman tree
    huffman_tree = {}

    while len(df) > 1:

        #first group the two least frequent word
        word1 = df.iloc[-1]
        word2 = df.iloc[-2]

        #Get cumulative frequency of the two least frequent words
        sum_freq = word1['count'] + word2['count']
        
        #Make a set of two least frequent words
        LeastWordsSet = np.array([word1['word'], word2['word']])
        LeastWordsSetWithFreqent = np.append(LeastWordsSet, sum_freq)

        #Create Key
        # Key1 = df.index[df['word'] == word1['word']].tolist()
        # Key2 = df.index[df['word'] == word2['word']].tolist()
        # UniqueKey = str(Key1[0]) +'_'+ str(Key2[0])
        UniqueKey = f"{word1['word']}_{word2['word']}"

        # Store parent-child relationship in Huffman tree
        huffman_tree[UniqueKey] = {"left": word1['word'], "right": word2['word'], "frequency": sum_freq}

   
        #dictionary of the words
        Set1 = {UniqueKey: sum_freq}

        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({'word': [UniqueKey], 'count': [sum_freq]})
        
        #remove the last two words
        df = df.drop(df.tail(2).index)

        df = pd.concat([df, new_row], ignore_index=True)


        # Sort the DataFrame by count
        df = df.sort_values(by='count', ascending=False)

        #print(f"Merged {word1['word']} and {word2['word']} -> {UniqueKey} ({sum_freq})")



    # print("\nFinal Huffman Tree Root Node:")
    # print(df.head())
    # print(f"Total Nodes in Final Tree: {len(df)}")



    # print(df.head())
    # print(len(df))

    #Print the Huffman tree dictionary
    # print("\nHuffman Tree Structure:")
    # for key, value in huffman_tree.items():
    #     print(f"{key}: {value}")

    # print how many nodes are in the tree

    return huffman_tree

def make_huffman_code(huffman_tree, prefix, node, huffman_code):
    if node not in huffman_tree:
        huffman_code[node] = prefix
        return

    left = huffman_tree[node]['left']
    right = huffman_tree[node]['right']

    make_huffman_code(huffman_tree, prefix + "0", left, huffman_code)
    make_huffman_code(huffman_tree, prefix + "1", right, huffman_code)

    return huffman_code







def Fano_prosedure(df = df.head(6)):
    """
    Finds the best split for Fano code
    """
    #sorting count

    # Create a new column to store the Fano code
    df['fano_code'] = ""

    # Calculate the cumulative frequency
    df['cumulative_freq'] = df['count'].cumsum()

    # Calculate the total frequency
    total_freq = df['count'].sum()


    # Calculate the average frequency
    df['Normalized_cf'] = df['cumulative_freq'] / total_freq

    min_index = (df['Normalized_cf']-0.5).abs().idxmin()
    # print(df)
    # print(f"min index:{min_index}")
    return min_index
    

fano_tree={}

def Fano_code(df, prefix):
   
    df = df.sort_values(by='count', ascending=False)


    #Split the dataframe
    smallest_index = Fano_prosedure(df)

    
    if len(df) > 1:
        left = df.iloc[:smallest_index+1].reset_index(drop=True)
        right = df.iloc[smallest_index+1:].reset_index(drop=True)
        

    else:
        word = df.iloc[0]['word']
        fano_tree[word] = prefix
        return 

    
    Fano_code(left, prefix + "0")
    Fano_code(right, prefix + "1")

    
    return fano_tree




def get_probability_for_word(word, df = df):
    """
    Get the probability of a word in the DataFrame
    """

    word = str(word)
    word_row = df[df['word'] == word]
    if len(word_row) == 0:
        return 0
    return word_row['count'].values[0] / df['count'].sum()




word_probs = {word: get_probability_for_word(word) for word in df['word']}

# print(word_probs)

def get_steps_in_fano(word, fano_tree):
    """
    Get the number of steps in Fano code for a word
    """
    return len(fano_tree[word])


def average_code_length_fano(fano_tree, word_probs):
    """
    Calculate the average code length for Fano code
    """
    return sum([get_steps_in_fano(word, fano_tree) * word_probs[word] for word in word_probs])



def get_steps_in_huffman(word, huffman_tree):
    """
    Get the number of steps in Huffman code for a word
    """
    return len(huffman_tree[word])

def average_code_length_huffman(huffman_tree, word_probs):
    """
    Calculate the average code length for Huffman code
    """
    return sum([get_steps_in_huffman(word, huffman_tree) * word_probs[word] for word in word_probs])

huffman_tree = get_huffman_code(df)
root_node = list(huffman_tree.keys())[-1]

huffman_code = make_huffman_code(huffman_tree, "", root_node, {})

fano_code = Fano_code(df, prefix='')


Huffman_avg_steps = average_code_length_huffman(huffman_code, word_probs)
Fano_avg_steps = average_code_length_fano(fano_code, word_probs)

difference_in_approaches = abs(Fano_avg_steps - Huffman_avg_steps)

print(f"Average code length for Fano code: {Fano_avg_steps}")
print(f"Average code length for Huffman code: {Huffman_avg_steps}")
print(f"Difference in approaches: {difference_in_approaches}")


# compute shannon entropy

def FindH(word_probs):
    """
    Compute Shannon Entropy
    """
    H = 0
    for word in word_probs:
        H += word_probs[word] * Math.log2(word_probs[word])
    H = -H
    return H




H = FindH(word_probs)
print(f"Shannon Entropy: {H}")



# print(f"Average code length for Huffman code: {average_code_length_huffman(make_huffman_code(get_huffman_code()), word_probs)}")

# 



Average code length for Fano code: 9.689373530048671
Average code length for Huffman code: 9.66819374072265
Difference in approaches: 0.021179789326021492
Shannon Entropy: 9.641114291987398


## Twenty-Questions: Constrained Questions

This code computes the word embeddings for the top 3000 words using the imported glove model. the distance measure is based on cosine similarity.

In [ ]:
#######################################################################################################################
##### a)
#######################################################################################################################

# Load the data from the CSV file
file_path = './data/unigram_freq.csv'
df = pd.read_csv(file_path, keep_default_na=False)

# Ttrim to the top 3000 most frequent words
df = df.sort_values(by='count', ascending=False).head(3000)

# Load the GloVe word embeddings model
model = api.load('glove-wiki-gigaword-50')

# dictionary to store the normalized embeddings for the words
word_embeddings = {}
missing_words = []

# For each word in the DataFrame, we will attempt to get its embedding
for word in df['word']:
    if word in model:
        embedding = model[word]
        # Normalize the embedding to have unit norm (i.e. project onto the unit sphere)
        norm = np.linalg.norm(embedding)
        if norm != 0:
            word_embeddings[word] = embedding / norm
        else:
            missing_words.append(word)
    else:
        missing_words.append(word)

print(f"Embeddings found for {len(word_embeddings)} out of {len(df)} words.")
print(f"Example missing words: {missing_words[:10]}")  # printing some of the missing words

# we create a DataFrame that contains only the words with available embeddings
df_embedded = df[df['word'].isin(word_embeddings.keys())].reset_index(drop=True)
print("DataFrame of words with embeddings:")
print(df_embedded.head())

# the word_embeddings dictionary has the normalized embeddings for the words, with df_embedded being the corresponding DataFrame.




Now, we use the embedding model to project the top 3000 words onto the unit sphere by normalizing their embeddings. Words that cannot be embedded by our choice of embedding model are discarded. Similarly as previously done, the prior probabilities is calulated using the frequencies of the words in the list.

We iteratively identify a hyperplane that divided the embedded words into two sets, and use this for the iterative procedure of finding the unknown word.

In [ ]:

#######################################################################################################################
##### b)
#######################################################################################################################


# --------------------------------------------------------------------
# 1. Project words onto the unit sphere and compute priors
# --------------------------------------------------------------------
# df_embedded have the top 3,000 words with embeddings.
# word_embeddings is a dictionary mapping each word to its normalized embedding.
total_count = df_embedded['count'].sum()
df_embedded = df_embedded.copy()  # create a local copy to avoid modifying the original
df_embedded['prob'] = df_embedded['count'] / total_count  # weight words by their frequency (prior probability)
word_probs = dict(zip(df_embedded['word'], df_embedded['prob']))

# this function is used to sample a random unit vector (used for candidate hyperplanes)
def sample_random_unit_vector(dim):
    v = np.random.randn(dim)
    return v / np.linalg.norm(v)

# --------------------------------------------------------------------
# 2. Iteratively identify a hyperplane that splits the current probability mass aprpox. 50/50
# --------------------------------------------------------------------
def find_best_hyperplane(candidates, word_embeddings, word_probs, num_candidates=100):
    dim = len(next(iter(word_embeddings.values())))
    best_n = None
    best_diff = float('inf')
    for _ in range(num_candidates):
        # this samples a random unit vector
        n = sample_random_unit_vector(dim)
        # here we compute the probability mass on the "positive" side (i.e. where dot product >= 0)
        pos_mass = sum(word_probs[w] for w in candidates if np.dot(word_embeddings[w], n) >= 0)
        diff = abs(pos_mass - 0.5)  # we want a split as close as possible to 50/50
        if diff < best_diff:
            best_diff = diff
            best_n = n
    return best_n

# --------------------------------------------------------------------
# 3, 4, and 5. Ask the question, drop wrong-side words, and iterate until resolution
# --------------------------------------------------------------------
def simulate_constrained_question_procedure(df_embedded, word_embeddings, word_probs):
    # Create the list of candidate words from our DataFrame (in 1. we already handled the projection and priors)
    candidates = list(df_embedded['word'])
    
    # we then sample an unknown word from the candidates using the prior probabilities
    words = np.array(candidates)
    probs = np.array([word_probs[w] for w in candidates])
    unknown = np.random.choice(words, p=probs)
    print("Unknown word (drawn from prior):", unknown)
    
    steps = 0
    # Set a variable to track if we are not reducing the candidate set
    stagnation_count = 0
    # Define a threshold for how many iterations with no reduction we'll tolerate
    STAGNATION_THRESHOLD = 5  # you can adjust this as needed
    # we continue the process until we have only one candidate left
    while len(candidates) > 1:
        steps += 1
        
        # we find a hyperplane that best splits the probability mass among the current candidates
        n = find_best_hyperplane(candidates, word_embeddings, word_probs, num_candidates=100)
        
        # we check if the unknown word is on the same side as the hyperplane
        unknown_side = np.dot(word_embeddings[unknown], n) >= 0
        
        # we update the candidate list based on the side of the hyperplane
        if unknown_side:
            new_candidates = [w for w in candidates if np.dot(word_embeddings[w], n) >= 0]
        else:
            new_candidates = [w for w in candidates if np.dot(word_embeddings[w], n) < 0]
        
        # Debug output to show how the probability mass is split.
        pos_mass = sum(word_probs[w] for w in candidates if np.dot(word_embeddings[w], n) >= 0)
        neg_mass = 1 - pos_mass
        print(f"Step {steps}: {len(candidates)} candidates; split: {pos_mass:.3f} vs. {neg_mass:.3f}.")


        # we check if the candidate set has shrunk significantly.
        if len(new_candidates) < len(candidates):
            stagnation_count = 0  # Reset if we made progress
        else:
            stagnation_count += 1  # No reduction detected
        
        # Fallback: if we have several iterations without reduction, choose the highest-probability candidate
        if stagnation_count >= STAGNATION_THRESHOLD:
            print("Candidate set did not shrink after several iterations. Using fallback selection.")
            chosen_candidate = max(candidates, key=lambda w: word_probs[w])
            print("Fallback candidate chosen:", chosen_candidate)
            return chosen_candidate
        
        # Updatate of the candidate list with the words that remain after dropping the wrong side.
        candidates = new_candidates
        print(f"After step {steps}, {len(candidates)} candidates remain.\n")
    
    print("Final candidate:", candidates[0])
    print("Total questions asked:", steps)


#a run of the procedure
# simulate_constrained_question_procedure(df_embedded, word_embeddings, word_probs)

## comments: in some runs of the simulation, we get a heavy-tailed distributino of the word ferquencies.
## therefore, we added a check in the iterative loop, such that if after an iteration the candidate set hasn’t reduced in size, we break the loop and choose the candidate with the highest prior probability.
## The fallback mechanism is a way to handle cases where the hyperplane splits are not effective in reducing the candidate set.



# Modified simulation function that returns the number of steps (questions) taken
def simulate_constrained_question_procedure_return_steps(df_embedded, word_embeddings, word_probs, stagnation_threshold=5, num_candidates=100):
    # Start with all candidate words
    candidates = list(df_embedded['word'])
    # Sample an unknown word using the prior probabilities
    words = np.array(candidates)
    probs = np.array([word_probs[w] for w in candidates])
    unknown = np.random.choice(words, p=probs)
    
    steps = 0
    stagnation_count = 0  # tracks consecutive iterations with no reduction in candidate count
    
    while len(candidates) > 1:
        steps += 1
        n = find_best_hyperplane(candidates, word_embeddings, word_probs, num_candidates=num_candidates)
        # Determine the side of the hyperplane where the unknown word lies
        unknown_side = np.dot(word_embeddings[unknown], n) >= 0
        # Partition candidates based on the hyperplane's side
        if unknown_side:
            new_candidates = [w for w in candidates if np.dot(word_embeddings[w], n) >= 0]
        else:
            new_candidates = [w for w in candidates if np.dot(word_embeddings[w], n) < 0]
        
        # Check if the candidate set is reduced
        if len(new_candidates) < len(candidates):
            stagnation_count = 0
        else:
            stagnation_count += 1
        
        # Fallback: if no reduction for several iterations, select the highest-probability candidate
        if stagnation_count >= stagnation_threshold:
            chosen_candidate = max(candidates, key=lambda w: word_probs[w])
            candidates = [chosen_candidate]
            break
        
        candidates = new_candidates
        
    return steps
    





We noticed after some runs of the procedure that the we got a heavy-tailed distribution of the word probabilities. Therefore, we added a check in the iterative loop, such that if after an iteration the candidate set hasn’t reduced in size, we break the loop and choose the candidate with the highest prior probability. The fallback mechanism is a way to handle cases where the hyperplane splits are not effective in reducing the candidate set.




Now, we simulate this procedure over multiple runs to estimate the expected number of questions one should ask to resolve the unknown word.

In [ ]:

#######################################################################################################################
##### c)
#######################################################################################################################

# Run the simulation multiple times to estimate the expected number of questions
num_trials = 100  # we adjust this number as needed
steps_list = []
for i in range(num_trials):
    steps = simulate_constrained_question_procedure_return_steps(df_embedded, word_embeddings, word_probs)
    steps_list.append(steps)

avg_steps = np.mean(steps_list)
std_steps = np.std(steps_list)
print(f"Average number of questions over {num_trials} trials: {avg_steps:.2f} ± {std_steps:.2f}")

# Compute Shannon entropy of the prior distribution as a lower bound
def shannon_entropy(word_probs):
    H = 0
    for p in word_probs.values():
        if p > 0:
            H -= p * np.log2(p)
    return H

H = shannon_entropy(word_probs)
print(f"Shannon entropy (lower bound on questions): {H:.2f}")



We computed the Shannon entropy to be 9.64.

After 10 simluation runs of the procedure, we found out that the constrained procedure is taking, on average, around 105 questions (with a high variance). This isexpected when using a restricted set of linear questions based on word embeddings, epsecially when the prior is heavily skewed. The optimal Shannon bound questions is only achievable if one designs an  ideal decision tree, which is not possible with the given constraints. The large gap between them indicated thhe cosntraiend procedure and the ideal procedure shows that the constrained procedure is much less efficient.  The high variance suggests that some trials resolve relatively quickly, while others stall, which is likely due to the heavy-tailed nature of the word frequencie.

A potential scheme that could be better than our approach here is to use the covariance structure of the candidate embeddings to choose hyperplanes along the direction of the greatest variance, which could yield a more balanced split.

## Kalman: Noisy Answers